In [1]:
import json
import os
from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *
import numpy as np
import SimpleITK as sitk
import csv
import shutil

In [ ]:
#Script to write csv to use in Deep Learning 

In [3]:
json_path = '/media/deeplearning/Elements/RELEVANCE_Valid_DICOM/relevance_data_dernier.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

cpt = 0
for serie in dataset : 
    if 'PET_Screening' in serie : 
        cpt += 1

print(cpt)
print(len(dataset))

36
36


In [4]:
data = []
for row in dataset : 
    if 'PET_Screening' in row : 
        data.append(row)

In [5]:
len(data)

36

In [6]:
#liste of generated nifti by "nifti_builder_from_json" script 

nifti_directory = '/media/deeplearning/Elements/RELEVANCE_NIFTI'
liste_nifti = os.listdir(nifti_directory)
no_nifti = []
for nifti in liste_nifti : 
    if "nifti" not in nifti : 
        no_nifti.append(nifti)

for fold in no_nifti : 
    liste_nifti.remove(fold)

In [7]:
len(liste_nifti)

70

In [8]:
#liste = []
#type_ = 'PET_Screening'
for row in data : 
    #subliste = []
    #subliste.append(row[0]) #patient id 
        #add other informations if need 

    #study_uid = row[1]
    #subliste.append(study_uid)

    subsubliste = []
    for nifti in liste_nifti : 
        if row[-3] in nifti: 
            subsubliste.append(nifti)
    if len(subsubliste) == 2 : #or == 3 if there is a mask 
        subsubliste = sorted(subsubliste)
        row.append(subsubliste[0])
        row.append(subsubliste[1])
            #subliste.append(subsubliste[2]) #add it if there is a mask 
    else : 
        pass
    #subliste.append(type_)
    #liste.append(subliste)



In [9]:
data[10]

['/media/deeplearning/Elements/RELEVANCE_Valid_DICOM/52011101831102/PET_Screening/1.2.840.113619.2.290.3.2831182250.47.1369721662.966.3/1.2.840.113619.2.290.3.2831182250.47.1369721662.966.3',
 'CT',
 '/media/deeplearning/Elements/RELEVANCE_Valid_DICOM/52011101831102/PET_Screening/1.2.840.113619.2.290.3.2831182250.47.1369721663.231/1.2.840.113619.2.290.3.2831182250.47.1369721663.231',
 'PT',
 '1.2.840.113619.2.290.3.2831182250.47.1369721662.961',
 'PET_Screening',
 '52011101831102',
 '1.2.840.113619.2.290.3.2831182250.47.1369721662.961_nifti_CT.nii',
 '1.2.840.113619.2.290.3.2831182250.47.1369721662.961_nifti_PT.nii']

In [10]:
#clean serie : must have patient ID, study_UID, CT_path, PET_path, MASK_path if pet0, and type "PET" (0,2,4)
completed_liste = []
for serie in data : 
    if len(serie) == 9 : #== 10 if Mask_path 
        completed_liste.append(serie)

In [35]:
#here check for double : if there are 2 patient with same ID, or same STUDY_UID, delete the other(s) serie(s)

dataset = []
dataset.append(completed_liste[0])
for i in range(1, len(completed_liste)) :
    if completed_liste[i] not in dataset : 
        dataset.append(completed_liste[i])

In [11]:
print("nombre de study : ", len(completed_liste))

nombre de study :  35


In [12]:
completed_liste[0]

['/media/deeplearning/Elements/RELEVANCE_Valid_DICOM/52011101841108/PET_Screening/1.3.12.2.1107.5.1.4.1007.30000013061907282506200012391/1.3.12.2.1107.5.1.4.1007.30000013061907282506200012391',
 'CT',
 '/media/deeplearning/Elements/RELEVANCE_Valid_DICOM/52011101841108/PET_Screening/1.3.12.2.1107.5.1.4.1007.30000013061907284295300020931/1.3.12.2.1107.5.1.4.1007.30000013061907284295300020931',
 'PT',
 '1.3.12.2.1107.5.1.4.1007.30000013061907243685900000017',
 'PET_Screening',
 '52011101841108',
 '1.3.12.2.1107.5.1.4.1007.30000013061907243685900000017_nifti_CT.nii',
 '1.3.12.2.1107.5.1.4.1007.30000013061907243685900000017_nifti_PT.nii']

In [13]:
#create folder to save nifti

for serie in completed_liste: 
    patient_id = serie[6]
    pet_type = serie[5]
    nifti_ct = serie[7]
    nifti_pet = serie[8]
    #nifti_mask = serie[5] #if pet0

    folder = '/media/deeplearning/Elements/RELEVANCE_NIFTI'+'/'+patient_id

    if not os.path.exists(folder):
        os.makedirs(folder)
        print(folder, 'created')
    else : print(folder, "Dossier patient déjà crée")

    subfolder = folder+'/'+pet_type
    if not os.path.exists(subfolder) :
        os.makedirs(subfolder)
        print(subfolder, 'created')
    else : print(subfolder, "Déjà creer => already folder for ", pet_type)

    ct_path = '/media/deeplearning/Elements/RELEVANCE_NIFTI'+'/'+nifti_ct
    pt_path = '/media/deeplearning/Elements/RELEVANCE_NIFTI'+'/'+nifti_pet
        #mask_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'+'/'+nifti_mask

    new_ct_path = subfolder+'/'+nifti_ct
    new_pet_path = subfolder+'/'+nifti_pet
    #new_mask_path = subfolder+'/'+nifti_mask
    try : 

        os.rename(ct_path, subfolder+'/'+nifti_ct)
        os.rename(pt_path, subfolder+'/'+nifti_pet)
        #os.rename(mask_path, subfolder+'/'+nifti_mask)


        serie[7] = new_ct_path
        serie[8] = new_pet_path 
        #serie[4] = new_mask_path
    except Exception as err : 
        print(err)


/media/deeplearning/Elements/RELEVANCE_NIFTI/52011101841108 created
/media/deeplearning/Elements/RELEVANCE_NIFTI/52011101841108/PET_Screening created
/media/deeplearning/Elements/RELEVANCE_NIFTI/52011101731105 created
/media/deeplearning/Elements/RELEVANCE_NIFTI/52011101731105/PET_Screening created
/media/deeplearning/Elements/RELEVANCE_NIFTI/52011101731107 created
/media/deeplearning/Elements/RELEVANCE_NIFTI/52011101731107/PET_Screening created
/media/deeplearning/Elements/RELEVANCE_NIFTI/52011101811103 created
/media/deeplearning/Elements/RELEVANCE_NIFTI/52011101811103/PET_Screening created
/media/deeplearning/Elements/RELEVANCE_NIFTI/52011101811104 created
/media/deeplearning/Elements/RELEVANCE_NIFTI/52011101811104/PET_Screening created
/media/deeplearning/Elements/RELEVANCE_NIFTI/52011101811106 created
/media/deeplearning/Elements/RELEVANCE_NIFTI/52011101811106/PET_Screening created
/media/deeplearning/Elements/RELEVANCE_NIFTI/52011101811107 created
/media/deeplearning/Elements/REL

In [21]:
completed_liste[0]

['/media/deeplearning/Elements/RELEVANCE_Valid_DICOM/12011101251120/PET_Screening/1.3.12.2.1107.5.1.4.11061.30000013080608134926500014351/1.3.12.2.1107.5.1.4.11061.30000013080608134926500014351',
 'CT',
 '/media/deeplearning/Elements/RELEVANCE_Valid_DICOM/12011101251120/PET_Screening/1.3.12.2.1107.5.1.4.11061.30000013080608151906200020007/1.3.12.2.1107.5.1.4.11061.30000013080608151906200020007',
 'PT',
 '1.2.840.113711.93413733.10800.397477827.26.2116281012.10',
 'PET_Screening',
 '12011101251120',
 '/media/deeplearning/Elements/RELEVANCE_NIFTI/12011101251120/PET_Screening/1.2.840.113711.93413733.10800.397477827.26.2116281012.10_nifti_CT.nii',
 '/media/deeplearning/Elements/RELEVANCE_NIFTI/12011101251120/PET_Screening/1.2.840.113711.93413733.10800.397477827.26.2116281012.10_nifti_PT.nii']

In [14]:
#write csv , add NIFTI_MASK column if there is a mask 
nifti_directory = '/media/deeplearning/Elements/RELEVANCE_NIFTI'
filename = 'RELEVANCE_NIFTI_SCREENING_v3.csv'


csv_path= '/media/deeplearning/Elements/RELEVANCE_NIFTI/RELEVANCE_NIFTI_SCREENING_v2.csv'
dataset_1 = []
with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    dataset_1 = []
    for row in reader :
        dataset_1.append(row)
        
del dataset_1[0] #enlever première ligne



with open(os.path.join(nifti_directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT ID", "STUDY UID", "PET", "NIFTI_CT", "NIFTI_PET"])
    for serie in dataset_1 : 
        csv_writer.writerow([serie[0], serie[1], serie[2], serie[3], serie[4]])
    for serie in completed_liste: 
        csv_writer.writerow([serie[6], serie[4], serie[5], serie[7], serie[8]])
